In [ ]:
using CairoMakie;
using CSV, DataFrames

using LilGuys
using Arya
import DensityEstimators as DE

In [ ]:
using StatsBase

In [ ]:
using PythonCall

In [ ]:
agama = pyimport("agama")

## File loading

In [ ]:
potential = agama.Potential("../agama/potentials/EP2020.ini")

In [ ]:
disk_poetntial = agama.Potential(potential[1], potential[2], potential[0])

In [ ]:
model_dir = ENV["DWARFS_ROOT"] * "/analysis/sculptor/1e7_V31_r3.2/orbit_mean/"

In [ ]:
starsfile = "../stars/plummer_rs0.20/probabilities_stars.hdf5"

In [ ]:
prob_df = LilGuys.read_hdf5_table(joinpath(model_dir, starsfile))

In [ ]:
issorted(prob_df.index)

In [ ]:
out = Output(model_dir, weights=prob_df.probability)

In [ ]:
r_max = 200

In [ ]:
bins = LinRange(-r_max, r_max, 1000 + 1)

## Stellar disk image

In [ ]:
Nb = length(bins)

In [ ]:
ρ_min = pyconvert(Float64, potential[3].density([r_max, r_max, r_max]))

In [ ]:
Σ_min = pyconvert(Float64, sum(potential[3].density([fill(r_max, Nb) fill(r_max, Nb) bins])))

In [ ]:
ρ_min * length(bins)

In [ ]:
function sum_density(disk_potential, bins; cutoff=40, cutoffstrength=3)
    bin_mids = midpoints(bins)
    Nb = length(bin_mids)
    
    Σ_disk = Matrix{Float64}(undef, Nb, Nb)
    for i in 1:Nb
        print("$i/$Nb\r")
        
        x = fill(bin_mids[i], Nb*Nb)
        y = vcat([fill(a, Nb) for a in bin_mids]...)
        z = repeat(bin_mids, Nb)
        r = @. sqrt(x^2 + y^2 + z^2)

        # NOTE--this should be more formalized
        pos = [x z y]
        rho = disk_poetntial.density(pos)
        rho = pyconvert(Array{Float64}, rho)
        if cutoff !== nothing
            rho = @. rho * exp( -(r / cutoff)^cutoffstrength) 
        end
        
        for j in 0:(Nb-1)
            Σ = sum(rho[1+j*Nb:Nb*(j+1)])
            Σ_disk[i, j+1] = Σ

        end
    end

    return Σ_disk
end
        

In [ ]:
function normalized_density(x, h=0.03)
    x = asinh.(x ./ h)
    xm, xh = extrema(x[isfinite.(x)])
    return (x .- xm) ./ (xh .- xm)
end

In [ ]:
function normalized_density2(x, h=0.03)
    x = log10.(x .+ h)
    xm, xh = extrema(x[isfinite.(x)])
    return (x .- xm) ./ (xh .- xm)
end

In [ ]:
color_0 = Arya.COLORS[7] 

In [ ]:
function transparency_map(x) 
    x = x
    Makie.RGBAf(color_0.r, color_0.g, color_0.b * x, x)
end

In [ ]:
Sigma_disk = sum_density(disk_poetntial, bins)

In [ ]:
Sigma_disk_nocut = sum_density(disk_poetntial, bins, cutoff=nothing)

In [ ]:
image(extrema(bins), extrema(bins), transparency_map.(normalized_density(Sigma_disk)),
    axis=(; limits=200 .* (-1, 1, -1, 1), aspect=DataAspect(), backgroundcolor=:black))

In [ ]:
image(extrema(bins), extrema(bins), transparency_map.(normalized_density(Sigma_disk_nocut)),
    axis=(; limits=200 .* (-1, 1, -1, 1), aspect=DataAspect(), backgroundcolor=:black))

In [ ]:
image(extrema(bins), extrema(bins), transparency_map.(normalized_density2(Sigma_disk, Σ_min)),
    axis=(; limits=200 .* (-1, 1, -1, 1), aspect=DataAspect(), backgroundcolor=:black))

In [ ]:
fig, ax = FigAxis(
    yscale=log10,
    limits=(nothing, nothing, 1, 1e6)
)

stephist!(vec(asinh.(Sigma_disk_nocut,)))
stephist!(vec(asinh.(Sigma_disk,)))
stephist!(vec(log10.(Sigma_disk .+ Σ_min)))
stephist!(vec(log10.(Sigma_disk_nocut .+ Σ_min)))
#
fig

In [ ]:
image(extrema(bins), extrema(bins), transparency_map.(normalized_density2(Sigma_disk_nocut, 10Σ_min)),
    axis=(; limits=200 .* (-1, 1, -1, 1), aspect=DataAspect(), backgroundcolor=:black))

## Animate dark matter

In [ ]:
function get_xy(out, idx)
    # shortcut for hdf5
    idx -= 1
    x = out.h5file["snap$idx/PartType1/Coordinates"][2, :]
	y = out.h5file["snap$idx/PartType1/Coordinates"][3, :]
    return x, y
end

In [ ]:
function project_points(x, y, bins)
    h1 = fit(Histogram, (x, y), (bins, bins) )
end

In [ ]:
@time x, y = get_xy(out, 1)

In [ ]:
@time h1 = project_points(x, y, bins)

In [ ]:
x, y = get_xy(out, 212)
h_end = project_points(x, y, bins)

In [ ]:
colormax = log10(maximum(h1.weights))
colorrange=(0, colormax)

In [ ]:
function make_frame(h, colorrange=colorrange)
    fig = Figure(figure_padding=(0,0,0,0), size=(200,200)) # px scale is x5 so this works for 1000 bins
    ax = Axis(fig[1, 1],
        aspect=DataAspect(),
    )
    

    p = image!(extrema(bins), extrema(bins), log10.(h.weights), colorrange=colorrange)

    image!(extrema(bins), extrema(bins), transparency_map.(normalized_density(Sigma_disk)))
    hidespines!(ax)
    resize_to_layout!(fig)
    hidedecorations!(ax)
    lines!([-175, -125], [-175, -175], color=:grey)
    text!(-175, -175, text="50 kpc", color=:grey, font=font, fontsize=7.5)

    return Makie.FigureAxisPlot(fig, ax, p)
end

In [ ]:
font = "/astro/dboyea/fonts/Arev.ttf"

In [ ]:
# example last frame

make_frame(h1)

In [ ]:
# example last frame

make_frame(h_end)

In [ ]:
h = project_points(x, y, bins)

In [ ]:
idxs = eachindex(out)

In [ ]:
animation_dir = "./figures/sculptor_dm_ani/"

In [ ]:
mkpath(animation_dir)

In [ ]:
function animate(out, bins, animation_dir)
    idx = eachindex(out)
    for i in idx
        print("animating $i / $(idx[end])\r")
        x, y = get_xy(out, i)
        h = project_points(x, y, bins)
        fig, ax, p = make_frame(h)
        Makie.save(joinpath(animation_dir, "frame_$i.png"), fig)
    end
end

In [ ]:
animate(out, bins, animation_dir)

## Animate stars

In [ ]:
idxs = eachindex(out)

In [ ]:
function get_xy(out, idx)
    # shortcut for hdf5
    snap = out[idx]
    x = snap.positions[2, :]
    y = snap.positions[3, :]
    w = snap.weights
    return x, y, w
end

In [ ]:
function project_points(x, y, w, bins)
    h1 = fit(Histogram, (x, y), weights(w), (bins, bins), )
end

In [ ]:
@time x, y, w = get_xy(out, 1)

In [ ]:
@time h = project_points(x, y, w, bins)

In [ ]:
x, y, w = get_xy(out, 212)
h_end = project_points(x, y, w, bins)

In [ ]:
sum(out.weights)

In [ ]:
Ntot = 5e6

In [ ]:
log10(Ntot)

In [ ]:
colormax = log10(maximum(h.weights))
colorrange = (-log10(Ntot), colormax) # from a single star to the maximum

In [ ]:
function make_frame(h, colorrange=colorrange)
    fig = Figure(figure_padding=(0,0,0,0), size=(200,200)) # px scale is x5 so this works for 1000 bins
    ax = Axis(fig[1, 1],
        aspect=DataAspect(),
    )
    

    p = image!(extrema(bins), extrema(bins), log10.(h.weights), colormap=Arya.get_arya_cmap(), colorrange=colorrange)
    image!(extrema(bins), extrema(bins), transparency_map.(normalized_density(Sigma_disk)))
    
    hidespines!(ax)
    resize_to_layout!(fig)
    hidedecorations!(ax)
    lines!([-175, -125], [-175, -175], color=:grey)
    text!(-175, -175, text="50 kpc", color=:grey, font=font, fontsize=7.5)

    return Makie.FigureAxisPlot(fig, ax, p)
end

In [ ]:
@time make_frame(h)

In [ ]:
@time make_frame(h_end)

In [ ]:
animation_dir = "./figures/sculptor_stars_ani/"

In [ ]:
mkpath(animation_dir)

In [ ]:
function animate(out, bins, animation_dir)
    idx = eachindex(out)
    for i in idx
        print("animating $i / $(idx[end])\r")
        x, y, w = get_xy(out, i)
        h = project_points(x, y, w, bins)
        fig, ax, p = make_frame(h)
        Makie.save(joinpath(animation_dir, "frame_$i.png"), fig)
    end
end

In [ ]:
animate(out, bins, animation_dir)

In [ ]:
Ntot = 5e6 # conservatively large estimate for number of stars

In [ ]:
sum(out.weights)

In [ ]:
log10(maximum(h1.weights))

In [ ]:
x, y, w = get_xy(out, 1)
h1 = project_points(x, y, w*Ntot, bins)

In [ ]:
log10(maximum(h1.weights))

In [ ]:
x, y, w = get_xy(out, 213)
h = project_points(x, y, w*Ntot, bins)

fig, ax, p = make_frame(h, (0, log10(maximum(h1.weights))) )
Makie.save(animation_dir * "/test.png", fig)

In [ ]:
log10(maximum(h1.weights / Ntot))

In [ ]:
x, y, w = get_xy(out, 213)
h = project_points(x, y, w, bins)

fig, ax, p = make_frame(h, (-6.7, log10(maximum(h1.weights / Ntot)) ))
Makie.save(animation_dir * "/test2.png", fig)

## Animate density profile

In [ ]:
function density_axis(gs; kwargs...)
    ax = Axis(gs;
        xlabel="log r / kpc",
        ylabel=L"\log\rho",
        limits=(-2, 2, -10, 1),
        kwargs...
    )

    return ax
end

In [ ]:
function density_axis()
    fig = Figure()
    ax = density_axis(fig[1, 1])

    return fig, ax
end

In [ ]:
times = out.times[profiles.snapshot_index] * lguys.T2GYR

In [ ]:
fig, ax = density_axis()
i = 1
lines!(profiles[i].log_r, log10.(profiles[i].rho))
fig

In [ ]:
fig, ax = density_axis()
i = length(times)
p = lines!(profiles[i].log_r, log10.(profiles[i].rho))
fig

In [ ]:


fig, ax = density_axis()
framerate=30
i = 1

p = lines!(profiles[i].log_r, log10.(profiles[i].rho))

record(fig, "sculptor_dm_density.mp4", 2:length(times), framerate = framerate) do i
    ax.title = string(times[i])
    
    prof = profiles[i]
    p[1] = [Makie.Point2f(x, y) for (x,y) in zip(prof.log_r, log10.(prof.rho))]
    println(i)
end

fig

## Animate orbit

In [ ]:
function r_t_axis()
    fig = Figure()
    ax = Axis(fig[1,1],
        xlabel="time / Gyr",
        ylabel=L"$r_\textrm{mw}$ / kpc",
    )

    return fig, ax
end

In [ ]:
function r_t_axis(gs)
    ax = Axis(gs,
        xlabel="time / Gyr",
        ylabel=L"$r_\textrm{mw}$ / kpc",
    )

    return ax
end

In [ ]:
r_mean = lguys.calc_r(out.x_cen)

In [ ]:
fig, ax = r_t_axis()
lines!(out.times * lguys.T2GYR, r_mean)
vlines!([NaN])
fig

In [ ]:
function rho_orbit_axis()
    fig = Figure(size=(700, 300))
    ax_rho = density_axis(fig[1, 1])
    ax_r = r_t_axis(fig[1, 2])

    ax_r.limits=(0, 10, 0.9minimum(r_mean), 1.1maximum(r_mean))

    colsize!(fig.layout, 1, Auto(1))
    colsize!(fig.layout, 2, Auto(0.5))

    return fig, ax_rho, ax_r
end

In [ ]:
rho_orbit_axis()[1]

In [ ]:
fig, ax, ax_r = rho_orbit_axis()
framerate=30

i = 1

p_rho = lines!(ax, profiles[i].log_r, log10.(profiles[i].rho))
orbit_points = Observable(Point2f[(out.times[1] * lguys.T2GYR, r_mean[1])])


p_r_break = vlines!(ax, [NaN], color=:black, alpha=0.3, linestyle=:dash)
p_r = lines!(ax_r, orbit_points)

r_last = Observable(r_mean[1])
r_last_last = Observable(r_last[])
t_last_peri = Observable(NaN)

record(fig, "sculptor_dm_density.mp4", 2:length(times), framerate = framerate) do i    
    prof = profiles[i]
    
    p_rho[1] = [Makie.Point2f(x, y) for (x,y) in zip(prof.log_r, log10.(prof.rho))]

    idx = profiles.snapshot_index[i]
    r = r_mean[idx]
    new_point = Point2f(times[i], r)
    orbit_points[] = push!(orbit_points[], new_point)

    if r_last_last[] > r_last[] && r_last[] < r_mean[idx]
        t_last_peri[] = times[i-1]
    end

    #todo oom but could be better
    σv = prof.v_circ_max * lguys.V2KMS / 3
    
    if !isnan(t_last_peri[])
        dt = times[i] - t_last_peri[]
        p_r_break[1] = [log10(σv * dt)] # approx
    end

    r_last_last[] = r_last[]
    r_last[] = r_mean[idx]
    println(i)

end

fig

## Animate phase space